In [15]:
!wget https://www.dropbox.com/scl/fi/bffhejvlmhbtto9v33cd5/model.zip?rlkey=1pcbdbovpg5p4g8vexmko1289&dl=0

--2023-12-11 19:20:04--  https://www.dropbox.com/scl/fi/bffhejvlmhbtto9v33cd5/model.zip?rlkey=1pcbdbovpg5p4g8vexmko1289
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb58635dc7eb0365bcca2553c0b.dl.dropboxusercontent.com/cd/0/inline/CJNPQ-9FAAmRCUX0Qx_vR_wrN27jtLpOQOTWHwqLepzTTMo_3KRTge-oYZb9tFNbj0QlZeScq7j-QOEOhx-Tmsk40qPHmT3sH-ovgQiPp48lsJLsjgGWUCquNHPi5aRreTpfF-ev941MRPxfdhy5Wz9x/file# [following]
--2023-12-11 19:20:04--  https://ucb58635dc7eb0365bcca2553c0b.dl.dropboxusercontent.com/cd/0/inline/CJNPQ-9FAAmRCUX0Qx_vR_wrN27jtLpOQOTWHwqLepzTTMo_3KRTge-oYZb9tFNbj0QlZeScq7j-QOEOhx-Tmsk40qPHmT3sH-ovgQiPp48lsJLsjgGWUCquNHPi5aRreTpfF-ev941MRPxfdhy5Wz9x/file
Resolving ucb58635dc7eb0365bcca2553c0b.dl.dropboxusercontent.com (ucb58635dc7eb0365bcca2553c0b.dl.dropboxusercontent.com)... 162.125.65.15, 2620:

In [16]:
!unzip -q model*.zip*

In [17]:
!mv /content/model_* /content/model
!rm -r /content/model*.zip*

In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
import keras_cv
from tensorflow import keras
import cv2
from matplotlib import pyplot as plt
import numpy as np

Using TensorFlow backend


In [ ]:
def resize_img_without_saving(img_path, input_size):
  my_image = plt.imread(img_path)

  if max(my_image.shape[0], my_image.shape[1]) > input_size:
    if my_image.shape[0] >= my_image.shape[1]:
      height = input_size
      width = int(my_image.shape[1] * input_size / my_image.shape[0])
    else:
      width = input_size
      height = int(my_image.shape[0] * input_size / my_image.shape[1])
    my_image = cv2.resize(my_image, (width, height), interpolation=cv2.INTER_LINEAR)

  return my_image


def pad_img_without_saving(my_image, input_size):

  pad_width = input_size - my_image.shape[0]
  pad_height = input_size - my_image.shape[1]
  my_padded_image = np.pad(my_image,
                           ((0, pad_width), (0, pad_height), (0, 0)),
                           'constant')
  return my_padded_image

In [ ]:
def plot_image_with_tlbr(image, box, show_label=True, confidence=None):

  _, axs = plt.subplots(nrows=1, ncols=1, figsize=(18, 4), squeeze=False)

  axs[0, 0].imshow(image)
  if box is not None:
    class_idx = 1
    xmin = box[0]
    ymin = box[1]
    xmax = box[2]
    ymax = box[3]
    w = xmax - xmin
    h = ymax - ymin
    axs[0, 0].add_patch(plt.Rectangle((xmin, ymin), w, h, fill=False, linewidth=2))
    if show_label:
      label = '{}'.format("logo")
      if confidence is not None:
        label += ' {:.2f}'.format(confidence)
      axs[0, 0].text(xmin, ymin, label, size='large', bbox={'alpha': 1.0})

In [ ]:
def load_model():
  dependencies = {'bounding_box_format': "xyxy"}
  loaded_model = keras.models.load_model('/content/model', custom_objects=dependencies, compile=False)
  return loaded_model

In [ ]:
model = load_model()

In [38]:
def detect_unibo_logo(image_file_path):
  # original image
  img = plt.imread(image_file_path)
  height, width = img.shape[:2]
  # resize
  input_size = 512
  img_resize = resize_img_without_saving(image_file_path, input_size)
  height_resize, width_resize = img_resize.shape[:2]
  # padding
  img_pad = pad_img_without_saving(img_resize, input_size)
  # bath
  img_batch = np.expand_dims(img_pad, axis=0)
  # predict
  predict = model.predict(img_batch)
  # result
  i = 0
  for conf in predict['confidence']:
    if conf.any() != -1:
      result = predict['boxes'][0][i]
    i+=1
  if any(-1 == b for b in result):
    result = None

  if result is not None:
    result[0] = result[0] * (width / width_resize)
    result[1] = result[1] * (height / height_resize)
    result[2] = result[2] * (width / width_resize)
    result[3] = result[3] * (height / height_resize)
    result = np.rint(result).astype(int).tolist()
  return result